### Kaggle link: https://www.kaggle.com/code/noobhocai/e2eqa/notebook

In [2]:
!pip install pyspark
!pip install pyarrow
!pip install nltk
!pip install tdqm
!pip install rank_bm25
# !pip install pandarallel
!pip install pyvi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317146 sha256=932384efffaa19a196f9bb66e53de64eb278df10f2ddf3186116516b1fff67b5
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
  Preparing metadata (setup.py) ... done
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1322 sha256=98dae54af0bf0a85639c88412f77b6a49d485bbc930a83a31edfb2dbc709a9ff
  Stored in directory: /root/.cache/pip/wheels/37/31/b8/7b711038035720ba0df14376af06e5e76b9bd61759c861ad92
Successfully built tdqm
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 57.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 55.0 MB/s eta 0:00:00


In [3]:
import pyspark
print(pyspark.__version__)

3.4.0


In [4]:
import json
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
import pyspark.pandas as ps
import re
import nltk
nltk.download('punkt')
from nltk import word_tokenize as lib_tokenizer 
from pyvi.ViTokenizer import tokenize
import string
from rank_bm25 import BM25Okapi
# from pandarallel import pandarallel

import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import pyspark as pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, lit, lower
from pyspark.sql.types import ArrayType, StringType, FloatType, IntegerType
MAX_MEMORY = '15G'
# Initialize a spark session.
# spark.stop()
conf = pyspark.SparkConf().setMaster("local[*]") \
        .set('spark.executor.heartbeatInterval', 10000) \
        .set('spark.network.timeout', 10000) \
        .set("spark.core.connection.ack.wait.timeout", "3600") \
        .set("spark.executor.memory", MAX_MEMORY) \
        .set("spark.driver.memory", MAX_MEMORY) \
        .set("spark.driver.maxResultSize", "4g") \
        .set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark = SparkSession.builder.appName("Pyspark").config(conf=conf).getOrCreate()
spark

/opt/conda/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/19 04:47:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# pandarallel.initialize(progress_bar=True, use_memory_fs=False)

In [6]:
data = json.load(open("/kaggle/input/e2eqa-wiki-zalo-ai/e2eqa-trainpublic_test-v1/e2eqa-train+public_test-v1/zac2022_train_merged_final.json"))
data = [x for x in data['data'] if x["category"] == "FULL_ANNOTATION" and "wiki/" in x["answer"]]

In [7]:
df_wiki = spark.read.json(path = "/kaggle/input/e2eqa-wiki-zalo-ai/wikipedia_20220620_cleaned/wikipedia_20220620_cleaned.jsonl")
# df_wiki = df_wiki.cache().pandas_api()
type(df_wiki)

pyspark.sql.dataframe.DataFrame

In [8]:
df_wiki.show(5)

+---+--------+--------------------+--------------------+----------------+--------------------+
| id|   revid|                text|           timestamp|           title|                 url|
+---+--------+--------------------+--------------------+----------------+--------------------+
|  2|68591979|Trang Chính\n\n<t...|2022-05-12T12:46:53Z|     Trang Chính|https://vi.wikipe...|
|  4|67988747|Internet Society\...|2022-01-20T07:59:10Z|Internet Society|https://vi.wikipe...|
| 13|68660631|Tiếng Việt\n\nTiế...|2022-05-29T03:42:42Z|      Tiếng Việt|https://vi.wikipe...|
| 24|68482118|Ohio\n\nOhio (viế...|2022-04-17T08:15:22Z|            Ohio|https://vi.wikipe...|
| 26|68738039|California\n\nCal...|2022-06-16T15:27:07Z|      California|https://vi.wikipe...|
+---+--------+--------------------+--------------------+----------------+--------------------+
only showing top 5 rows



In [9]:
df_wiki.createOrReplaceTempView('df_wiki_temp')
spark.catalog.listTables()

[Table(name='df_wiki_temp', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [10]:
def get_topk(query, topk=100):
    tokenized_query = query.split()
    tfidf_query = dictionary.doc2bow(tokenized_query)
    scores = bm25_index[tfidf_query]
    top_n = np.argsort(scores)[::-1][:topk]
    titles = [df_wiki.filter(col("id") == i).select("title").collect()[0][0] for i in top_n]
    texts = [df_wiki.filter(col("id") == i).select("text").collect()[0][0] for i in top_n]
    scores = [scores[i] for i in top_n]
    result = "\n".join([f"{title}\t{text}\t{score}" for title, text, score in zip(titles, texts, scores)])
    return result

get_topk_udf = udf(get_topk, StringType())

dict_map = {}

def word_tokenize(text): 
    global dict_map
    words = text.split() 
    words_norm = [] 
    for w in words: 
        if w not in dict_map: 
            dict_map[w] = ' '.join(lib_tokenizer(w)).replace('``', '"').replace("''", '"') 
        words_norm.append(dict_map[w]) 
    return words_norm 

word_tokenize_udf = udf(word_tokenize, ArrayType(StringType()))

def strip_answer_string(text):
    text = text.strip() 
    while text[-1] in '.,/><;:\'"[]{}+=-_)(*&^!~`': 
        if text[0] != '(' and text[-1] == ')' and '(' in text: 
            break 
        if text[-1] == '"' and text[0] != '"' and text.count('"') > 1: 
            break 
        text = text[:-1].strip() 
    while text[0] in '.,/><;:\'"[]{}+=-_)(*&^!~`': 
        if text[0] == '"' and text[-1] != '"' and text.count('"') > 1: 
            break 
        text = text[1:].strip() 
    text = text.strip() 
    return text 

strip_answer_string_udf = udf(strip_answer_string, StringType())

def strip_context(text):
    text = text.replace('\n', ' ') # thay kí tự kết thúc bằng khoảng trắng
    text = re.sub(r'\s+', ' ', text) # thay khoảng trắng lớn hơn 1 kí tự thành khoảng trắng 1 kí tự
    text = text.strip() # xoá khoảng trắng ở đầu và cuối 
    return text 

strip_context_udf = udf(strip_context, StringType())

def post_process(x):
    x = x.lower()
    x = " ".join(word_tokenize(strip_context(x))).strip()
    x = x.replace("\n"," ")
    x = "".join([i for i in x if i not in string.punctuation])
    x = " ".join(x.split()[:128])
    return x 

post_process_udf = udf(post_process, StringType())

In [11]:
df_wiki = df_wiki.withColumn("title_lower", post_process_udf(col("title")))
df_wiki = df_wiki.withColumn("text_lower", post_process_udf(col("text")))
df_wiki.show(5)

+---+--------+--------------------+--------------------+----------------+--------------------+----------------+--------------------+
| id|   revid|                text|           timestamp|           title|                 url|     title_lower|          text_lower|
+---+--------+--------------------+--------------------+----------------+--------------------+----------------+--------------------+
|  2|68591979|Trang Chính\n\n<t...|2022-05-12T12:46:53Z|     Trang Chính|https://vi.wikipe...|     trang chính|trang chính templ...|
|  4|67988747|Internet Society\...|2022-01-20T07:59:10Z|Internet Society|https://vi.wikipe...|internet society|internet society ...|
| 13|68660631|Tiếng Việt\n\nTiế...|2022-05-29T03:42:42Z|      Tiếng Việt|https://vi.wikipe...|      tiếng việt|tiếng việt tiếng ...|
| 24|68482118|Ohio\n\nOhio (viế...|2022-04-17T08:15:22Z|            Ohio|https://vi.wikipe...|            ohio|ohio ohio viết tắ...|
| 26|68738039|California\n\nCal...|2022-06-16T15:27:07Z|      Califor

In [12]:
df_wiki.explain()

== Physical Plan ==
*(1) Project [id#8, revid#9, text#10, timestamp#11, title#12, url#13, pythonUDF0#144 AS title_lower#85, pythonUDF1#145 AS text_lower#94]
+- BatchEvalPython [post_process(title#12)#84, post_process(text#10)#93], [pythonUDF0#144, pythonUDF1#145]
   +- FileScan json [id#8,revid#9,text#10,timestamp#11,title#12,url#13] Batched: false, DataFilters: [], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/kaggle/input/e2eqa-wiki-zalo-ai/wikipedia_20220620_cleaned/wikip..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<id:string,revid:string,text:string,timestamp:string,title:string,url:string>




### Đọc file __stopwords__

In [13]:
filename = '/kaggle/input/e2eqa-wiki-zalo-ai/external/stopwords.csv'
stop = pd.read_csv(filename, sep="\t", encoding='utf-8')
list_stopwords = stop['stopwords']
list_stopwords

0         mà_thôi
1          tuy_có
2         đâu_như
3       quay_bước
4         hỏi_lại
          ...    
1937       cái_họ
1938          hết
1939          lại
1940      thế_thế
1941      dần_dần
Name: stopwords, Length: 1942, dtype: object

---
---

In [14]:
# Tokenize text into words
def tokenizer(text):
    tokens = [word for word in tokenize(text).split() if word not in list_stopwords]
    return tokens

tokenize_udf = udf(tokenizer, ArrayType(StringType()))
df_wiki = df_wiki.withColumn("words", tokenize_udf(col("text_lower")))
df_wiki.select("words").show(5)

+--------------------+
|               words|
+--------------------+
|[trang, chính, te...|
|[internet, societ...|
|[tiếng, việt, tiế...|
|[ohio, ohio, viết...|
|[california, cali...|
+--------------------+
only showing top 5 rows



### Tần suất các từ trong texts

In [15]:
from pyspark.sql import functions as F
# Increase the number of partitions
# df_wiki = df_wiki.repartition(100)

# Cache the DataFrame in memory
df_wiki.cache()

# Perform the computation
result = (
    df_wiki
    .select(F.explode("words").alias("word"))
    .groupBy("word")
    .count()
    .orderBy(F.desc("count"))
)

# Show the results
result.show()

+--------+-------+
|    word|  count|
+--------+-------+
|      là|1858454|
|    loài|1385128|
|     một|1365013|
|   trong|1046730|
|     năm|1034325|
|    được| 923562|
|      họ| 846094|
|     này| 726077|
|  bullet| 671970|
|     của| 656976|
|      có| 602435|
|      và| 578809|
|khoa_học| 484960|
|đầu_tiên| 381971|
|       ở| 373949|
|     hoa| 352879|
|   thuộc| 339118|
|   người| 321510|
|     các| 308467|
|thực_vật| 287653|
+--------+-------+
only showing top 20 rows



---
---

#### Map title with answer

In [16]:
from pyspark.sql import Window
# add index column
df_wiki = df_wiki.rdd.zipWithIndex().toDF(["data", "index"]).select("data.*", "index")

In [17]:
from pyspark.sql.functions import trim, lower, regexp_replace

# Create a dictionary mapping titles to indices
title2idx = (
  df_wiki
  .select(trim("title").alias("title"), "index")
  .groupBy("title")
  .agg({"index": "first"})
  .rdd
  .collectAsMap()
)

# Loop over data and check if answer is in the dictionary
for x in data:
    answer = x['answer'].replace("wiki/", "").replace("_", " ")
    # Check if the cleaned up title is in the dictionary
    if answer not in title2idx.keys():
        print(answer)
        for key, val in title2idx.items():
            if answer.lower() == key.lower():
                x['answer'] = "wiki/" + key.replace(" ", "_")
    

Cù Lao Chàm
nhà Hán
bán đảo Ả Rập


In [18]:
l = []
for x in data:
    l.append(x['answer'])
len(set(l)), len(l)

(1849, 4608)

#### Lọc các stopwords ra khỏi answer

In [19]:
# from pyspark.ml.feature import StopWordsRemover
# # Tokenize and remove stop words
# remover = StopWordsRemover(inputCol="title_lower", outputCol="tokens")
# tokenized_df = remover.transform(df_wiki.select("title_lower"))

# # Convert tokens to a list of strings
# to_list = udf(lambda x: x, ArrayType(StringType()))
# tokenized_df = tokenized_df.withColumn("token_list", to_list("tokens"))

# # Compute TF-IDF vectors
# cv = CountVectorizer(inputCol="token_list", outputCol="raw_features")
# cv_model = cv.fit(tokenized_df)
# featurized_df = cv_model.transform(tokenized_df)
# idf = IDF(inputCol="raw_features", outputCol="features")
# idf_model = idf.fit(featurized_df)
# rescaled_df = idf_model.transform(featurized_df)

# # Convert features column to VectorUDT
# to_vector = udf(lambda x: x, VectorUDT())
# rescaled_df = rescaled_df.withColumn("features", to_vector("features"))

# # Compute BM25Okapi score
# bm25 = BM25Okapi(rescaled_df.select("features").rdd.flatMap(lambda x: x))
# bm25

In [20]:
docs = df_wiki.select('title_lower').toPandas()
tokenized_corpus = [[word for word in doc.lower().split() if word not in list_stopwords]
                    for doc in docs['title_lower']
                   ] #simple tokenier
bm25_title = BM25Okapi(tokenized_corpus)

In [21]:
df = pd.DataFrame({
    "query": [post_process(x["short_candidate"]).lower().split() for x in data]
})
df = spark.createDataFrame(df)
df.show(5)

+--------------------+
|               query|
+--------------------+
|[costa, rica, ice...|
|       [núi, elbrus]|
|        [alexandria]|
|          [lê, chân]|
|                 [ý]|
+--------------------+
only showing top 5 rows



In [22]:
df.select('query').show()

+--------------------+
|               query|
+--------------------+
|[costa, rica, ice...|
|       [núi, elbrus]|
|        [alexandria]|
|          [lê, chân]|
|                 [ý]|
|       [phan, thiết]|
|      [xã, nhơn, lý]|
|            [google]|
|    [tỉnh, gia, lai]|
|  [tỉnh, quảng, nam]|
|   [vua, khải, định]|
|[theo, thể, lục, ...|
|   [trần, duy, hưng]|
|       [suối, tranh]|
|[chùa, làng, vũ, ...|
|[nguyễn, phú, trọng]|
|              [pháp]|
|   [tỉnh, an, giang]|
|    [tỉnh, bắc, kạn]|
|   [nguyễn, văn, tý]|
+--------------------+
only showing top 20 rows



In [23]:
# Define UDF to compute topk titles for a query
def get_topk(query, topk=10):
    scores = bm25_title.get_scores(query)
    top_titles = list(map(int, np.argsort(scores)[::-1][:topk]))
    return top_titles

get_topk_udf = udf(get_topk, ArrayType(IntegerType()))

# Add topk titles column to DataFrame
df = df.withColumn("top_n", get_topk_udf(df["query"]))
df.show(5)

+--------------------+--------------------+
|               query|               top_n|
+--------------------+--------------------+
|[costa, rica, ice...|[6771, 25055, 585...|
|       [núi, elbrus]|[11092, 1268687, ...|
|        [alexandria]|[54875, 223778, 5...|
|          [lê, chân]|[25316, 17081, 39...|
|                 [ý]|[4452, 46524, 114...|
+--------------------+--------------------+
only showing top 5 rows



In [24]:
candidate_ids = []
true_ids = []
top_n_df = df.select("top_n").toPandas()
for i, x in enumerate(data):
    true_title = x["answer"].replace("_"," ").replace("wiki/","").strip()
    top_n = top_n_df.loc[i, "top_n"].tolist() 
    true_idx = title2idx[true_title]
    true_ids.append(true_idx)
    candidate_ids.append(top_n)

In [25]:
for x, y in zip(candidate_ids, true_ids):
    if y not in x:
        x[-1] = y

In [26]:
# Change to list of row
rows = df_wiki.collect()

In [27]:
df = pd.DataFrame()
questions = []
answers = []
titles = []
candidates = []
labels = []
groups = []
for idx, (sample, true_idx, candidate_idxs) in enumerate(zip(data, true_ids, candidate_ids)):
    assert true_idx in candidate_idxs
    question = sample['question']
    answer = sample['short_candidate']
    title = [rows[i]['title'] for i in candidate_idxs]
    candidate = [rows[i]['text'] for i in candidate_idxs]
    label = [1 if x == true_idx else 0 for x in candidate_idxs]
    
    questions.extend([question,]*len(candidate_idxs))
    answers.extend([answer,]*len(candidate_idxs))
    groups.extend([idx,]*len(candidate_idxs))
    titles.extend(title)
    candidates.extend(candidate)
    labels.extend(label)
    
df["question"] = questions 
df["answer"] = answers 
df["title"] = titles 
df["candidate"] = candidates 
df["label"] = labels 
df["group"] = groups 
df.candidate = df.candidate.apply(lambda x: " ".join(x.strip().split()))

In [28]:
df.head()

,question,answer,title,candidate,label,group
0,Đất nước nào không có quân đội,"Costa Rica, Iceland, Panama, Micronesia, Quần ...",Costa Rica,Costa Rica Costa Rica (Phiên âm: Cô-xta Ri-ca)...,0,0
1,Đất nước nào không có quân đội,"Costa Rica, Iceland, Panama, Micronesia, Quần ...",Quần đảo Marshall,"Quần đảo Marshall Quần đảo Marshall, tên chính...",0,0
2,Đất nước nào không có quân đội,"Costa Rica, Iceland, Panama, Micronesia, Quần ...","Montezuma, Costa Rica","Montezuma, Costa Rica Montezuma là một thị xã ...",0,0
3,Đất nước nào không có quân đội,"Costa Rica, Iceland, Panama, Micronesia, Quần ...","Tamarindo, Costa Rica","Tamarindo, Costa Rica Tamarindo là một thị xã ...",0,0
4,Đất nước nào không có quân đội,"Costa Rica, Iceland, Panama, Micronesia, Quần ...",Micronesia,"Micronesia Micronesia (, ), còn gọi là Tiểu Đả...",0,0


In [30]:
df.to_csv("/kaggle/working/train_stage2_ranking.csv",index=False)

<a href="/kaggle/working/train_stage2_ranking.csv"> Download File </a>

In [33]:
!zip -r file.zip /kaggle/working/train_stage2_ranking.csv

  adding: kaggle/working/train_stage2_ranking.csv (deflated 70%)


In [36]:
!cd /kaggle && ls -halt working

total 507M
drwxr-xr-x 4 root root 4.0K Jun 19 05:38 .
-rw-r--r-- 1 root root 118M Jun 19 05:38 file.zip
-rw-r--r-- 1 root root 390M Jun 19 05:33 train_stage2_ranking.csv
drwxr-xr-x 2 root root 4.0K Jun 19 04:47 spark-warehouse
drwxr-xr-x 2 root root 4.0K Jun 19 04:45 .virtual_documents
drwxr-xr-x 5 root root 4.0K Jun 19 04:45 ..
---------- 1 root root  263 Jun 19 04:45 __notebook_source__.ipynb
